# Grouping Data with SQL

## Introduction

Just as with Pandas, we can use aggregate functions in SQL to assist with data manipulation. Sometimes you may wish to find the mean, median, min, or max of a column feature. For example, there could be a customer relational database that you've been working with and you may wonder if there are differences in overall sales across offices or regions.

## Objectives

You will be able to:

* Write queries with aggregate functions like `COUNT`, `MAX`, `MIN`, and `SUM`
* Create an alias for the return value of an aggregate function
* Use `GROUP BY` to sort the data sets returned by aggregate functions
* Compare aggregates using the `HAVING` clause

## Database Schema
<img src="images/Database-Schema.png">

In [2]:
import sqlite3
import pandas as pd

## Connecting to the Database

As usual, start by creating a connection to the database and instantiating a cursor object.

In [3]:
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## `GROUP BY` and Aggregate Functions

Lets start by looking at some `GROUP BY` statements to aggregate our data. The `GROUP BY` clause groups records into summary rows and returns one records for each group.
Typically, `GROUP BY`  also involves an aggregate function (`COUNT`, `AVG`, etc.). Lastly, `GROUP BY` can group by one or more columns.

In the cell below, we'll join the offices and employees tables in order to count the number of employees per city.

In [4]:
cur.execute("""SELECT city, COUNT(employeeNumber)
                      FROM offices
                      JOIN employees
                      USING(officeCode)
                      GROUP BY city
                      ORDER BY count(employeeNumber) DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,COUNT(employeeNumber)
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


## Aliasing

An Alias is a shorthand for a table or column name. Aliases reduce the amount of typing required to enter a query. Generally, complex queries with aliases are easier to read.
Aliases are useful with `JOIN`, `GROUP BY`, and aggregates (`SUM`, `COUNT`, etc.).
An Alias only exists for the duration of the query.

You can alias your `GROUP BY` by specifying the index of our selection order that we want to group by. This is simply written as `GROUP BY 1`, with the number "1" referring to the first column name that we are selecting.

Additionally, we can also rename our aggregate to a more descriptive name using the `AS` clause.

In [4]:
cur.execute("""SELECT city, COUNT(employeeNumber) AS numEmployees
               FROM offices
               JOIN employees
               USING(officeCode)
               GROUP BY 1
               ORDER BY numEmployees DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,numEmployees
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


## Other Aggregations

Aside from `COUNT()` some other useful aggregations include:
- `MIN()`
- `MAX()`
- `SUM()`
- `AVG()`

In [5]:
cur.execute("""SELECT customerName,
               COUNT(customerName) AS number_purchases,
               MIN(amount) AS min_purchase,
               MAX(amount) AS max_purchase,
               AVG(amount) AS avg_purchase,
               SUM(amount) AS total_spent
               FROM customers
               JOIN payments
               USING(customerNumber)
               GROUP BY 1
               ORDER BY SUM(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

98


,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
0,Euro+ Shopping Channel,13,116208.40,65071.26,55056.844615,715738.98
1,Mini Gifts Distributors Ltd.,9,101244.59,85410.87,64909.804444,584188.24
2,"Australian Collectors, Co.",4,44894.74,82261.22,45146.267500,180585.07
3,Muscle Machine Inc,4,20314.44,58841.35,44478.487500,177913.95
4,"Dragon Souveniers, Ltd.",4,105743.00,44380.15,39062.757500,156251.03


In [6]:
df.tail()

,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
93,Royale Belge,4,1128.20,1627.56,7304.295000,29217.18
94,Frau da Collezione,2,17746.26,7612.06,12679.160000,25358.32
95,Atelier graphique,3,14571.44,6066.78,7438.120000,22314.36
96,Auto-Moto Classics Inc.,3,5858.56,9658.74,7184.753333,21554.26
97,Boards & Toys Co.,2,3452.75,4465.85,3959.300000,7918.60


## The `HAVING` clause

Finally, we can also filter our aggregated views with the `HAVING` clause. The `HAVING` clause works similarly to the `WHERE` clause, except it is used to filter data selections on conditions **after** the `GROUP BY` clause. For example, if we wanted to filter based on a customer's last name, we would use the `WHERE` clause. However, if we wanted to filter a list of cities with at least 5 customers, we would using the `HAVING` clause. First, we would `GROUP BY` city and then use the `HAVING` clause, which will allow us to pass conditions on the result of this aggregation.

In [12]:
cur.execute("""SELECT city, COUNT(customerNumber) AS number_customers
               FROM customers
               GROUP BY 1
               HAVING COUNT(customerNumber)>=5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()
# so WHERE is for columns, HAVING is for aggregate funcs


2


,city,number_customers
0,Madrid,5
1,NYC,5


In [14]:
cur.execute('''SELECT * from customers''').fetchall()

[('103',
  'Atelier graphique',
  'Schmitt',
  'Carine ',
  '40.32.2555',
  '54, rue Royale',
  '',
  'Nantes',
  '',
  '44000',
  'France',
  '1370',
  '21000.00'),
 ('112',
  'Signal Gift Stores',
  'King',
  'Jean',
  '7025551838',
  '8489 Strong St.',
  '',
  'Las Vegas',
  'NV',
  '83030',
  'USA',
  '1166',
  '71800.00'),
 ('114',
  'Australian Collectors, Co.',
  'Ferguson',
  'Peter',
  '03 9520 4555',
  '636 St Kilda Road',
  'Level 3',
  'Melbourne',
  'Victoria',
  '3004',
  'Australia',
  '1611',
  '117300.00'),
 ('119',
  'La Rochelle Gifts',
  'Labrune',
  'Janine ',
  '40.67.8555',
  '67, rue des Cinquante Otages',
  '',
  'Nantes',
  '',
  '44000',
  'France',
  '1370',
  '118200.00'),
 ('121',
  'Baane Mini Imports',
  'Bergulfsen',
  'Jonas ',
  '07-98 9555',
  'Erling Skakkes gate 78',
  '',
  'Stavern',
  '',
  '4110',
  'Norway',
  '1504',
  '81700.00'),
 ('124',
  'Mini Gifts Distributors Ltd.',
  'Nelson',
  'Susan',
  '4155551450',
  '5677 Strong St.',
  '',
  '

## Combining the `WHERE` and `HAVING` clause
We can also use the `WHERE` and `HAVING` clauses in conjunction with each other for more complex rules.
For example, let's say we want a list of customers who have made at least 3 purchases of over 50K each.

In [13]:
cur.execute("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY 1
               HAVING count(amount) >= 3
               ORDER BY count(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

'''
we want list of customers with at least 3 purchases over 50k each

''

53


,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Anna's Decorations, Ltd",4
3,"Australian Collectors, Co.",4
4,Baane Mini Imports,4


In [9]:
df.tail()

,customerName,number_purchases_over_50K
48,"Stylish Desk Decors, Co.",3
49,Suominen Souveniers,3
50,"Toys of Finland, Co.",3
51,Toys4GrownUps.com,3
52,Vitachrome Inc.,3


## Summary

After this section, you should have a good idea of how to use aggregate functions, aliases and the having clause to filter selections.